# OPENAI LLM Baseline Code

In [1]:
# put this at the top of your script/module
from pathlib import Path
import sys

def add_root_for_utils(max_up=8, utils_dir_name="utils"):
    try:
        start = Path(__file__).resolve().parent   # 일반 .py 파일
    except NameError:
        start = Path.cwd().resolve()              # Jupyter/REPL 등

    p = start
    for _ in range(max_up + 1):
        if (p / utils_dir_name).is_dir():
            root = str(p)
            if root not in sys.path:
                sys.path.insert(0, root)
            return p
        if p.parent == p:
            break
        p = p.parent

    raise RuntimeError(f"상위 {max_up}단계 내에서 '{utils_dir_name}' 폴더를 찾지 못했어요.")

# 실행
add_root_for_utils(max_up=8)


WindowsPath('C:/Potenup/Drug-Detection-Chatbot')

In [2]:
from utils.llm_openai import chat, chat5

# 데이터 불러오기

In [3]:
import pandas as pd

df = pd.read_csv("C:\Potenup\Drug-Detection-Chatbot\data\\total_result_drug.csv")

In [4]:
df.head(1)

,C-Code,구분,제품명,복지부분류코드,volume,name,vol_value,vol_unit,성분/함량,첨가물,제형,성상,KPIC/ATC,구분(전문/일반),효능,용법,주의사항,저장방법,name_ko,name_en
0,K-000059,일반의약품,게루삼정 200T,234.0,200T,게루삼정,200.0,T,"['Calcium Carbonate P.P.T. 침강탄산칼슘 100mg', 'Dri...","['스테아르산마그네슘', '카르복시메틸셀룰로오스칼슘']",정제,흰색의 원형정제 백색의 원형 정제,"['소화기계질환', '소화성궤양 치료제', '제산제', '흡수성', '비흡수성', ...",일반,"위산과다, 속쓰림, 위부불쾌감, 위부팽만감, 체함, 구역, 구토, 위통, 신트림.","만 15세 이상 및 성인: 1회 2정, 1일 3회 만 15세 이상 및 성인: 1회 ...",1. 다음과 같은 사람은 이 약을 복용하지 말 것. 1) 투석요법을 받고 있는 환자...,"기밀용기, 실온(1~30℃)보관",게루삼정,Gelusam Tab


In [ ]:
df[df['제품명'].str.contains("타이레놀", na=False, regex=False)]['제품명']

111      타이레놀정500mg(성인용)
112        어린이용타이레놀정80mg
125           타이레놀정160mg
143    타이레놀8시간이알서방정650mg
315    우먼스타이레놀정500mg 10T
352        타이레놀콜드에스정 10T
Name: 제품명, dtype: object

In [44]:
data = ""
for i, row in list(df.iterrows()):
    data += f"({str(i)}, {row['제품명']})"

# 모델 예측하기

In [64]:
system_prompt = f"""
당신은 약 정보에 대한 CSV 데이터를 가지고 해당 데이터에서 검색을 하는 AI 입니다.
알약에 대해서 키워드 검색을 하면 해당 알약과 관련된 모든 알약의 index 값을 찾는 것이 목표입니다.
제품명에 해당 키워드를 포함하거나, 비슷한 경우 모두 index를 포함합니다.
[사용자 입력]
알약에 대한 키워드 검색

[사전 정보]
{data}

[출력 형식]
해당 제품명에 해당하는 제품 인덱스 리스트 ex) [0, 2],
"""

In [46]:
user = f"""
타이레놀
"""

In [65]:
answer = chat(system_prompt, user)
print(answer)

[111, 125]


# 결과

In [72]:
import json
answer_parser = json.loads(answer)

In [74]:
data = []
for index in answer_parser:
    data.append(df.iloc[index])

In [78]:
result_df = pd.DataFrame(data)
result_df

,C-Code,구분,제품명,복지부분류코드,volume,name,vol_value,vol_unit,성분/함량,첨가물,제형,성상,KPIC/ATC,구분(전문/일반),효능,용법,주의사항,저장방법,name_ko,name_en
111,K-004378,일반의약품,타이레놀정500mg(성인용),114.0,500mg,타이레놀정,500.0,mg,['Acetaminophen 아세트아미노펜 500mg'],"['분말셀룰로오스', '스테아르산마그네슘', '오파드라이흰색(YS-1-7027)',...",정제,흰색의 장방형 필름코팅정제,"['통증 질환', '비마약성 진통제', '중추성 진통제', 'p-aminopheno...",일반,"1. 주효능·효과 1. 주효능·효과 감기로 인한 발열 및 동통(통증), 두통, 신경...",만 12세 이상 소아 및 성인: 만 12세 이상 소아 및 성인: 1회 1~2정씩 1...,1. 경고 1) 매일 세잔 이상 정기적으로 술을 마시는 사람이 이 약이나 다른 해열...,"밀폐용기, 실온보관(1-30℃)",타이레놀정,mg Tylenol Tab
125,K-004799,일반의약품,타이레놀정160mg,114.0,160mg,타이레놀정,160.0,mg,['Acetaminophen 아세트아미노펜 500mg'],"['분말셀룰로오스', '스테아르산마그네슘', '오파드라이흰색(YS-1-7027)',...",정제,흰색의 장방형 필름코팅정제,"['통증 질환', '비마약성 진통제', '중추성 진통제', 'p-aminopheno...",일반,"1. 주효능·효과 1. 주효능·효과 감기로 인한 발열 및 동통(통증), 두통, 신경...",만 12세 이상 소아 및 성인: 만 12세 이상 소아 및 성인: 1회 1~2정씩 1...,1. 경고 1) 매일 세잔 이상 정기적으로 술을 마시는 사람이 이 약이나 다른 해열...,"밀폐용기, 실온보관(1-30℃)",타이레놀정,mg Tylenol Tab
